In [1]:
import sys
sys.path.insert(0,'data')

import config
import utils_db
import utils
import utils_storage
import multiprocessing,time
def upload_img_to_yandex_storage(i):
    url_storage = utils_storage.load_image_on_yandex_storage(name = i['image_id'],img_url = i['url_vk'])
    utils_db.add_url_storage_in_db(image_id = i['image_id'],url_storage = url_storage)
    
import logging
logger = logging.getLogger('vk_exception_h')
handler = logging.FileHandler('logs/uploader_to_yandex_storage_h.log',encoding = "UTF-8")
formatter = logging.Formatter('------------\n %(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter); logger.addHandler(handler)

In [ ]:
while True:
    try:
        st = time.time()
        #select img whith status 0
        img_to_upload = utils_db.select_img_to_yandex_storage_upload()
        #paralel download to yandex storage
        if len(img_to_upload) > 0:
            pool = multiprocessing.Pool(processes=min(10,len(img_to_upload)))
            pool.map(upload_img_to_yandex_storage, img_to_upload)
            pool.close()
        time.sleep(max(2 - (time.time()-st),0))
    except KeyboardInterrupt:
        break
    except Exception as e:
        logger.exception("error uploader_to_yandex_storage_h ")
        utils.send_me_error("error uploader_to_yandex_storage_h ", e)